<style>.md-sidebar--secondary {display: none !important;}</style>

# Custom Forecaster

In this example, we use a custom forecast algorithm, the "Olympic Average", 
implemented in Python.  In this forecast, the mean (mu) forecast for any
bucket is computed by taking the 26 historical values, discarding the highest
and lowest, and taking the mean of the remaining 24 values.

In [ ]:
import passengersim as pax
pax.versions()

We'll first run a simulation without the change as a baseline.  As this tutorial is 
meant as a technology demonstration and not for serious statistical analysis, we'll
only run one trial so it goes fast, and we won't worry about statistical validity.

In [ ]:
cfg = pax.Config.from_yaml([
    "network/08-untrunc-em.yaml",
])

In [ ]:
cfg.simulation_controls.num_trials = 1

In [ ]:
sim = pax.Simulation(cfg)
baseline_summary = sim.run()

In [ ]:
baseline_summary.fig_carrier_revenues()

Now we swap in our custom forecast algorithm.

In [ ]:
from passengersim.extra.olympic_forecast import OlympicForecastStep

In [ ]:
extra_config = """
rm_systems:
  rm_test2:
    availability_control: leg
    processes:
      DCP:
      - step_type: untruncation
        algorithm: em
        kind: leg
      - step_type: olympicforecast
      - step_type: emsr
        algorithm: emsrb
airlines:
  - name: AL1
    rm_system: rm_test2
  - name: AL2
    rm_system: rm_test1
"""

cfg = pax.Config.from_yaml([
    "network/08-untrunc-em.yaml",
    extra_config,
])
cfg.simulation_controls.num_trials = 1

In [ ]:
sim = pax.Simulation(cfg)

In [ ]:
summary = sim.run()

In [ ]:
summary.fig_carrier_revenues()

## Comparing Results

In [ ]:
from passengersim.contrast import Contrast

comp = Contrast(Baseline=baseline_summary, Olympic=summary)

In [ ]:
comp.fig_carrier_revenues()

In [ ]:
comp.fig_carrier_load_factors()

In [ ]:
comp.fig_fare_class_mix()

In [ ]:
comp.fig_bookings_by_timeframe(by_carrier="AL1", by_class=True, source_labels=True)

In [ ]:
comp.fig_leg_forecasts(by_flt_no=101, of=["mu", "sigma"])

In [ ]:
comp.fig_leg_forecasts(by_flt_no=111, of=["mu", "sigma"])